# MGMTMSA 403 Assignment 1: 
# Operating Room Scheduling Optimization

#### By Xiaoyi Wang

### Part A
<br> First, formulate a schedule that minimizes the total under-allocation of each department
to operating rooms. For example, General Surgery currently receives 48.4% of the total
operating room time, and should therefore receive 48.4% of the total operating room time in the
new schedule as well. In particular, the CEO has specified that under-allocation is to be avoided,
meaning a penalty should be incurred if a department is allocated less than its target %, but
there should be no penalty if a department is allocated more than its target % of operating
room time.

<br> Your model should produce a new schedule that minimizes the total under-allocation (on a
percentage basis). The reason for representing allocation on a percentage basis is that using
units of time is not equitable: For example, a loss of 1 hour per week is much more disruptive
to Oral Surgery (currently 10 hrs/week) than it is for General Surgery (92 hrs/week).

In [4]:
from gurobipy import *
import numpy as np

mod = Model()

tr = 213.5 #total hours

# Target allocated time avoiding under-allocation
t = [tr * x for x in [0.484, 0.042, 0.253, 0.074, 0.053, 0.095]]

# duration of the room avaliability from Monday to Friday
d = np.array([[9,9,9,9,7.5], [9,9,9,9,7.5], [9,9,9,9,7.5], [9,9,9,9,7.5], [9,8,8,8,6.5]])

# binary decision for every possibilities of i department gets j room on k weekday.
i = len(t)
j = len(d[0])
k = len(d)

x = mod.addVars(i, j, k, vtype = GRB.BINARY)

s = mod.addVars(i)

#Constraint
for i in range(6):
    mod.addConstr(t[i] - sum(sum(x[i,j,k] * d[j][k] for j in range(5)) for k in range (5)) <= s[i])

for j in range(5):
    for k in range(5):
        mod.addConstr(sum(x[i,j,k] for i in range(6)) <= 1)

pen_con = mod.addConstrs(s[i] >= 0 for i in range(6))


#Objective
mod.setObjective(sum(s[i] / t[i] for i in range(6)), GRB.MINIMIZE)
mod.update()
mod.optimize()

Using license file C:\Users\jason\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-12
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 37 rows, 156 columns and 312 nonzeros
Model fingerprint: 0x93771b6d
Variable types: 6 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)

Root relaxation: objective 2.066116e-03, 63 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

In [5]:
mod.objval

0.05190597648316861

### Part B
<br> Operating rooms Main-1, Main-2 will be located on the first floor of the new hospital, Main-3 and Main-4 will be on the second floor, and Main-5 will be located on the third floor. 

<br> To improve communication and mobility among department staff, the CEO has inquired whether it is possible to devise the schedule so that no department is split between two or more floors on the same day. For example, it is acceptable if a department is exclusively assigned to Main-1 on Monday and then Main-5 on Tuesday, but not acceptable if a department is assigned to Main-1 and Main-5 on the same day.

<br>Incorporate constraints into your base model from part a) to ensure that no department is allocated rooms on two different floors on the same day.

In [6]:
fmod = Model()

tr = 213.5 #total hours

# Target allocated time avoiding under-allocation
t = [tr * x for x in [0.484, 0.042, 0.253, 0.074, 0.053, 0.095]]

# duration of the room avaliability from Monday to Friday
d = np.array([[9,9,9,9,7.5], [9,9,9,9,7.5], [9,9,9,9,7.5], [9,9,9,9,7.5], [9,8,8,8,6.5]])

# binary decision for every possibilities of i department gets j room on k weekday.
i = len(t)
j = len(d[0])
k = len(d)

x = fmod.addVars(i, j, k, vtype = GRB.BINARY)

s = fmod.addVars(i)

f = fmod.addVars(i, j, 3)

#Constraint
for i in range(6):
    fmod.addConstr(t[i] - sum(sum(x[i,j,k] * d[j][k] for j in range(5)) for k in range (5)) <= s[i])

for j in range(5):
    for k in range(5):
        fmod.addConstr(sum(x[i,j,k] for i in range(6)) <= 1)

pen_con = fmod.addConstrs(s[i] >= 0 for i in range(6))

for i in range(6):
    for k in range(5):
        fmod.addConstr((2 * f[i, k, 0]) >= (x[i,0,k] + x[i,1,k]))
        fmod.addConstr((2 * f[i, k, 1]) >= (x[i,2,k] + x[i,3,k]))
        fmod.addConstr(f[i, k, 2] >= x[i,4,k])

for i in range(6):
    for k in range(5):
        fmod.addConstr(sum(f[i,k,l] for l in range(3)) <= 1)
    

#Objective
fmod.setObjective(sum(s[i] / t[i] for i in range(6)), GRB.MINIMIZE)
fmod.update()
fmod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 157 rows, 246 columns and 642 nonzeros
Model fingerprint: 0xfcb457da
Variable types: 96 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 96 rows and 90 columns
Presolve time: 0.00s
Presolved: 61 rows, 156 columns, 456 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)

Root relaxation: objective 1.580699e-01, 69 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       0.1580699    0.00000   100%     -    0s
     0     

In [7]:
fmod.Objval

0.15806994793581977